In [ ]:
! pip install dash-dangerously-set-inner-html

In [ ]:
# Global import

import datetime as dt

import pandas as pd
import plotly.graph_objects as go

from dash_mdc_neptune import themes

In [ ]:
# Global variables

TODAY = pd.to_datetime(dt.date.today(), utc=True)

CURVE_UID = "FWD_PWRTE_H_DA"
PUBLISHED_AT = TODAY - pd.tseries.offsets.Day(30)

# PAGE 1

In [ ]:
# Retrieve auctions

query_params = {
    "curve_uid": CURVE_UID,
    "curve_type": "FIXING",
    "published_at": PUBLISHED_AT,
}

ts_da = nptc.retrieve_time_series(**query_params)

In [ ]:
# Format auctions

df_da = pd.DataFrame(ts_da, columns=['price'])
df_da = df_da.reset_index().rename(columns={"index": "delivery_from"})

# Add auction date and period
df_da = npta.convert_utc_to_date_and_sp(
    df_da, timezone_str="Europe/Paris", frequency_timedelta=dt.timedelta(minutes=60)
)

# Add local delivery
df_da["delivery_from_local"] = df_da.delivery_from.dt.tz_convert("Europe/Paris")

## SECTION 1

### CARD 1

In [ ]:
# Extract latest auctions

df_latest_da = df_da.tail(24)
df_latest_da = df_latest_da[
    ["settlement_date", "settlement_period_id", "price", "delivery_from_local"]
]
df_latest_da = df_latest_da.rename(
    columns={
        "settlement_date": "auction date",
        "settlement_period_id": "auction period",
        "price": "€/MWh",
        "delivery_from_local": "local delivery",
    }
)

In [ ]:
# Format table

df_styler = df_latest_da.style
df_styler = df_styler.hide_index()
df_styler = df_styler.background_gradient(
    axis=0, gmap=df_latest_da["€/MWh"], cmap=themes.matplotlib.noos_diverging
)
df_styler = df_styler.format({"€/MWh": "{:,.2f}", "hour": "{:,.0f}"})
df_styler = df_styler.set_table_attributes("style='font-size: 0.8rem'")

## SECTION 2

In [ ]:
# Shared heatmap configs

GO_LAYOUT = go.Layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='local delivery date',
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='local delivery hour',
        )
    ),
    template="noos_watermark+noos_base+noos_colorscale+min_modebar+hoover_xunified",
)

GO_HEATMAP_KWARGS = {
    "colorscale": themes.NOOS_PALETTE_DIVERGING,
    "colorbar": {"title": "€/MWh"},
    "hovertemplate": (
        "delivery date: %{x:%Y-%m-%d}<br>"
        "delivery hour: %{y:.0f}<br>"
        "price: %{z:.2f}<extra></extra>"
    ), 
}

In [ ]:
# Extract heatmap data

df_heatmap_da = df_da.set_index("delivery_from_local")

df_heatmap_da["date"] = df_heatmap_da.index.date.astype("str")
df_heatmap_da["hour"] = df_heatmap_da.index.hour

### CARD 1

In [ ]:
# Weekly heatmap

wtd_date = PUBLISHED_AT - pd.tseries.offsets.Week(1)
df_wtd_da = df_heatmap_da[wtd_date.date().isoformat():]

wtd_data = go.Heatmap(
    z=df_wtd_da.price,
    x=df_wtd_da.date,
    y=df_wtd_da.hour,
    **GO_HEATMAP_KWARGS
)

wtd_fig = go.FigureWidget(data=wtd_data, layout=GO_LAYOUT)

### CARD 3

In [ ]:
# Monthly heatmap

mtd_date = PUBLISHED_AT - pd.tseries.offsets.Week(4)
df_mtd_da = df_heatmap_da[mtd_date.date().isoformat():]

mtd_data = go.Heatmap(
    z=df_mtd_da.price,
    x=df_mtd_da.date,
    y=df_mtd_da.hour,
    **GO_HEATMAP_KWARGS
)

mtd_fig = go.FigureWidget(data=mtd_data, layout=GO_LAYOUT)

### CARD 2

In [ ]:
# Quarterly heatmap

qtd_date = PUBLISHED_AT - pd.tseries.offsets.Week(12)
df_qtd_da = df_heatmap_da[qtd_date.date().isoformat():]

qtd_data = go.Heatmap(
    z=df_qtd_da.price,
    x=df_qtd_da.date,
    y=df_qtd_da.hour,
    **GO_HEATMAP_KWARGS
)

qtd_fig = go.FigureWidget(data=qtd_data, layout=GO_LAYOUT)

# DASH COMPONENTS

In [ ]:
import jupyter_dash

import dash_dangerously_set_inner_html as html
from dash import dcc

import dash_mdc_neptune as mdc

In [ ]:
table = html.DangerouslySetInnerHTML(df_styler.to_html())
table_section = mdc.Section(
    id="section-table",
    size=3,
    children=table,
    cards=[{"title": "Latest"}],
)

extras = {
    "style": {"height": "100%", "width": "100%"},
}
wtd_graph = dcc.Graph(
    id="weekly-graph",
    figure=wtd_fig,
    **extras
)
mtd_graph = dcc.Graph(
    id="monthly-graph",
    figure=mtd_fig,
    **extras
)
qtd_graph = dcc.Graph(
    id="quarterly-graph",
    figure=qtd_fig,
    **extras
)
graph_tab = mdc.Tab(
    id="tab-graph",
    children=[wtd_graph, mtd_graph, qtd_graph],
    tabs=[
        {"label": "Last week"},
        {"label": "Last month"},
        {"label": "Last quarter"},
    ],
)
graph_section = mdc.Section(
    id="section-graph",
    children=graph_tab,
    cards=[{"title": "To Date"}],
)

page = mdc.Page(children=[table_section, graph_section])
navbar = mdc.NavBar(title="Day-Ahead Auctions")

layout = mdc.Dashboard(children=[navbar, page])

In [ ]:
from jupyter_dash import JupyterDash


app = JupyterDash(__name__)
app.layout = layout

app.run_server(mode='external', host="0.0.0.0", port=8001, debug=False)